In [ ]:
# import modules
import dawdreamer as daw
import numpy as np
import os
from scipy.io.wavfile import write
from utils import *

In [ ]:
# define constants
SAMPLE_RATE = 22050
BLOCK_SIZE = 128 # Parameters will undergo automation at this buffer/block size.
SYNTH_NAME = "TAL-Uno"

import platform
if platform.system() == "Windows":
    SYNTH_PLUGIN = "C:/VSTPlugins/TAL-U-NO-LX-V2-64.dll"
    PRESET_FOLDER = "~/AppData/Roaming/ToguAudioLine/TAL-U-No-LX/presets"
elif platform.system() == "Darwin":
    SYNTH_PLUGIN = "/Library/Audio/Plug-Ins/VST3/TAL-U-NO-LX-V2.vst3"  # extensions: .dll, .vst3, .vst, .component
    PRESET_FOLDER = "~/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets"
else:
    raise ValueError("This notebook hasn't been tested on Linux yet.")

PRESET_EXT = ".pjunoxl"
SUSTAIN_DUR = 3  # in seconds
DURATION = 5  # in seconds
NOTE = 'A4'  # Western Scale
PRESET_FOLDER = os.path.expanduser(PRESET_FOLDER)

In [ ]:
# create a RenderEngine
engine = daw.RenderEngine(sample_rate=SAMPLE_RATE, block_size=BLOCK_SIZE)

# create the plugin projessor
plugin = engine.make_plugin_processor(SYNTH_NAME, SYNTH_PLUGIN)

In [ ]:
# randomly select a preset from the preset folder
preset_path = select_preset_path(PRESET_FOLDER,PRESET_EXT)

In [ ]:
# create JSON parameter map and save it
json_file_name = make_json_parameter_mapping(plugin, preset_path, verbose=False)

In [ ]:
# apply the synth preset settings to the synth plugin processor object
loaded_preset_synth = load_xml_preset(plugin, json_file_name)

In [ ]:
# convert the piano note to midi (0 to 127)
midi_note = piano_note_to_midi_note(NOTE)

# generate a sound using the plugin
velocity = 60
loaded_preset_synth.add_midi_note(midi_note, velocity, 0.0, SUSTAIN_DUR)

engine.load_graph([(loaded_preset_synth, [])])

# loaded_preset_synth.open_editor()
engine.render(DURATION)

audio = engine.get_audio()

In [ ]:
# make the diectory if it doesn't exist
os.makedirs('output', exist_ok=True)

# write the output to a WAV file
out_name = json_file_name.split(os.sep)[-1].split('json')[0]
write(f'output{os.sep}{out_name}-d{DURATION}-{NOTE}.wav', SAMPLE_RATE, audio[0,:])

# get the spectrogram
spec = audio2mel_spectrogram(audio_folder_path=f'output', plot_flag=True, zero_padding_factor=1, range_db=80,
                             gain_db=20, n_mels=128, f_max=4096)